# Data Science con Pandas 

## Datos del Plan Sumar: https://www.kaggle.com/c/eci2017-1
## Indicadores nutricionales
### BMIZ: body-mass-index-for-age-Z-scores (referencia estandarizada del BMI por edad y género).
### HAZ: height-for-age Z-scores (referencia estandarizada del la altura por edad y género).
### WAZ: weight-for-age Z-scores (referencia estandarizada del peso por edad y género).
## Otras variables
### individuo: ID asignado a cada individuo.
### bmi: bmi = peso / (altura^2).
### departamento_indec_id, departamento_lat, departamento_long 
### fecha_control, fecha_nacimiento, fecha_proximo_control, genero
### nombre_provincia: nombre de la provincia en donde se atendió el individuo
### nombre_region: nombre de la región en donde se atendió el individuo
### perimetro_encefalico: medición de perímetro encefálico obtenida en la atención (cm)
### peso: medición de peso obtenida en la atención (kg)
### talla: medición de altura obtenida en la atención (cm)
### provincia_indec_id: código de provincia (S = sí pertenece, N = no pertenece). Coincide con el código obtenido del INDEC.
### zona_rural: código que indica si el efector se encuentra en una zona rural.

## Las siguientes variables se encuentran disponibles únicamente en el conjunto de datos de entrenamiento.

### var_BMIZ: variación que tendrá BMIZ en la siguiente atención respecto al valor actual
### var_HAZ: variación que tendrá HAZ en la siguiente atención respecto al valor actual
### var_WAZ: variación que tendrá WAZ en la siguiente atención respecto al valor actual
### decae (Variable a estimar): Toma el valor "True" si al menos una de las siguientes condiciones (c1, c2 y c3) ocurre:
### c1 = (HAZ >= -1)  & (var_HAZ < 0)   & ((HAZ + var_HAZ) < -1)
### c2 = (WAZ >= -1)  & (var_WAZ < 0)  & ((WAZ + var_WAZ) < -1)
### c3 = (BMIZ >= -1) & (var_BMIZ < 0) & ((BMIZ + var_BMIZ) < -1)
### Es decir, si alguno de los indicadores cae por debajo del umbral de un desvío estándar del promedio, siendo que antes no lo estaba. En caso contrario asume el valor "False".

In [ ]:
# importo paquetes y funciones
from __future__ import division # para la division real
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

In [ ]:
# carga el archivo
data = pd.read_table("train_data.txt", sep=",")

In [ ]:
# muestralos primeros 10 registros
data.head(10)

In [ ]:
#devuelve  (# filas, # columnas)
data.shape

In [ ]:
#devuelve tamaño del data frame
data.size

# Selección de datos

In [ ]:
# Selecciona las columnas del Dataframe (y las guarda en la variable llamada "columnas")
coulumnas  = data.columns
coulumnas

In [ ]:
# Devuelve los indices del Dataframe
data.index

In [ ]:
# Devuelve una columna según su nombre (devuelve una Serie)
data["peso"]

In [ ]:
data.talla[0]

In [ ]:
# Seleccion de varias columnas (devuleve un Dataframe)
data[["individuo","peso","nombre_provincia"]]

In [ ]:
# Devuelve una fila por index (devuelve una Serie)
data.loc[10]

In [ ]:
# Devuelve varias filas por index (devuelve un Dataframe)
data.loc[[1000,2,76,23]]

In [ ]:
# Devuelve un elemento por index y columna 
data.loc[2512,"nombre_provincia"]

In [ ]:
# Devuelve elementos por indices 
data.iloc[15:20,0:5]

In [ ]:
data.iloc[1:10,:]

In [ ]:
data.iloc[10:,-4:]

## Selección a partir de los datos

In [ ]:
# serie con elementos booleanos 
criterio_de_filtrado = data.HAZ < -1
criterio_de_filtrado

In [ ]:
# devuelve las filas con True
data[criterio_de_filtrado]

In [ ]:
# and =  & 
# or  =  |
# not = ~
criterio_de_filtrado2 = (data["BMIZ"]< -1) | (data["HAZ"]< -1) | (data["WAZ"]< -1)
data.loc[criterio_de_filtrado2,"nombre_provincia"]

In [ ]:
# El método "value_counts()" devuelve las frecuencias de apariciones
data.loc[criterio_de_filtrado2,"nombre_provincia"].value_counts()

In [ ]:
data["nombre_provincia"].value_counts()

## Operaciones

In [ ]:
# cociente entre 2 Series
frac_riesgo_por_prov = data.loc[criterio_de_filtrado2,"nombre_provincia"].value_counts()/data["nombre_provincia"].value_counts()
frac_riesgo_por_prov

In [ ]:
# cociente entre Series y un real
frac_riesgo_por_prov*100

In [ ]:
# aplicacion de la función "round" a una serie
porcentage_riesgo_por_prov = np.round(frac_riesgo_por_prov*100,2)
porcentage_riesgo_por_prov

In [ ]:
# quiero reescribiro como srting y agregarle el signo "%"
# para eso defino la funcion porciento_a_texto()
def porciento_a_texto(x):
    return str(x)+"%"

porciento_a_texto(21.22)

In [ ]:
# no puedo aplicar la funcion "porciento_a_texto()"! 
# para eso esta apply()
porciento_a_texto(porcentage_riesgo_por_prov)

In [ ]:
# Aplica la funcion porciento_a_texto() a cada elemente de la serie
porcentage_riesgo_por_prov.apply(porciento_a_texto)

# Transformaciones sobre el dataset

In [ ]:
#Crea una copia del contenido del Dataframe
data2 = data.copy()

In [ ]:
# Agrega una nueva columna
data2["peso_sobre_talla"] = data2["peso"]/data2["talla"]
data2.columns

In [ ]:
# Elimina una columna (axis=0 refiere a filas, axis=1 refiere a columnas)
data2.drop("individuo",axis=1,inplace=True)
data2.columns

## Trabajo con fechas para calcular la edad

In [ ]:
data2['fecha_control'].head()

In [ ]:
# las fechas estan en formato string
data2['fecha_control'][0]

In [ ]:
# formatea la fecha
pd.to_datetime(data2['fecha_control'],format="%Y-%m-%d").head()

In [ ]:
# calcula la la edad a partir de la diferencia entre "fecha_control" y "fecha_nacimiento" 
days = pd.to_datetime(data2['fecha_control'],format="%Y-%m-%d")-pd.to_datetime(data2['fecha_nacimiento'],format="%Y-%m-%d")
days.head()

In [ ]:
# extrae el numero (int) de dias
days = days.apply(lambda x: x.days)
days.head()

In [ ]:
# crea una nueva columna en el dataframe
data2["days"]=days

# Exploración del dataset

In [ ]:
# años con los que se cuentan datos
control_year = pd.to_datetime(data2['fecha_control'],format="%Y-%m-%d").apply(lambda x: x.year)
control_year.value_counts()

In [ ]:
# Calcula la mediana de los indices por provincia
data2.groupby(by="nombre_provincia")["BMIZ", "HAZ", "WAZ"].median()

In [ ]:
# Calcula la mediana de los indices por departamento y los ordena segun "HAZ"
medianas_por_dep = data2.groupby(by="departamento_indec_id")["BMIZ", "HAZ", "WAZ"].median().sort_values(by="HAZ")
medianas_por_dep

In [ ]:
# departamento_indec_id = 469  <=>  Lincoln, Bs As
# departamento_indec_id = 270  <=>  Ezeiza, Bs As

In [ ]:
# numero de registros por departamento
registros_por_dep = data2["departamento_indec_id"].value_counts()
registros_por_dep.head()

In [ ]:
# agrego esta columna al dataframe de las medianas por depto. 
medianas_por_dep["registros_por_dep"] = registros_por_dep
medianas_por_dep

# Ploteos

In [ ]:
# Scatter plot
medianas_por_dep.plot.scatter(x="HAZ", y="registros_por_dep")

In [ ]:
medianas_por_dep.plot.scatter(x="HAZ", y="registros_por_dep")
plt.yscale("log") # cambio a escala logaritmica
plt.ylabel("Numero de registros") # cambio el texto del eje Y
plt.title("Datos del Plan Sumar")  # agrego un titulo
plt.savefig("numero_de_registros_VS_HAZ.png") # guardo la figura

# Relaciones entre variables

In [ ]:
sns.pairplot(data2,vars =["BMIZ","HAZ", "WAZ"],diag_kind="kde",markers=".",size=2)

In [ ]:
# calula las correlaciones entre los indicadores 
data2[["BMIZ","HAZ", "WAZ"]].corr(method='pearson')

In [ ]:
data2.plot.scatter(x='BMIZ',y='HAZ',c='WAZ',cmap='seismic')
plt.axvline(x=-1,color="black")
plt.axhline(y=-1,color="black")

# ¿Qué combinación variables resultan útiles para identificar un decaimiento?

In [ ]:
sns.pairplot(data2,vars =["BMIZ","HAZ"],diag_kind="kde",markers="+", hue="decae",plot_kws={'alpha':0.50})

### Identifico a "ojo" zonas con mayor concentración de decaimiento

In [ ]:
ax = data2[data2.decae==False].plot.scatter(x='BMIZ', y='HAZ', color='blue', label='no decae', alpha=0.5);
data2[data2.decae==True].plot.scatter(x='BMIZ', y='HAZ', color='green', label='decae', ax=ax, alpha=0.5);
plt.axhline(y=-3.5,color="black")
plt.axhline(y=1.5,color="black")

In [ ]:
ax = data2[data2.decae==False].plot.scatter(x='BMIZ', y='HAZ', color='blue', label='no decae', alpha=0.5);
data2[data2.decae==True].plot.scatter(x='BMIZ', y='HAZ', color='green', label='decae', ax=ax, alpha=0.5);
plt.axhline(y=-3.5,color="black")
plt.axhline(y=1.5,color="black")
plt.axhline(y=-1,color="black")
plt.vlines(-1,-3.5,-1,color="black")
plt.vlines(2,-3.5,-1,color="black")
plt.vlines(-2,-1,1.5,color="black")
plt.vlines(2.5,-1,1.5,color="black")

In [ ]:
zona1 = (-3.5<data2.HAZ) & (data2.HAZ<-1) & (-1<data2.BMIZ) & (data2.BMIZ<2)
zona2 = (-1<data2.HAZ) & (data2.HAZ<1.5) & (-2<data2.BMIZ) & (data2.BMIZ<2.5)
zona_de_decaimiento = (zona1) | (zona2)

In [ ]:
from sklearn.metrics import confusion_matrix
CM = confusion_matrix(data2.decae,zona_de_decaimiento, labels=[True,False])
CM

In [ ]:
df_CM = pd.DataFrame(CM,index=["decae","no decae"],columns=["ident. decae","ident. no decae"])
df_CM

In [ ]:
# identificamos aleatoreamente igual numero de decaimientos
random_decae = zona_de_decaimiento.copy()
np.random.shuffle(random_decae)
random_decae[:10]

In [ ]:
# Matriz de confucion con las identificaciones shufleadas
CM_random = confusion_matrix(data2.decae,random_decae, labels=[True,False])
df_CM_random = pd.DataFrame(CM_random,index=["decae","no decae"],columns=["ident. decae","ident. no decae"])
df_CM_random

In [ ]:
print "identificacion original"
df_CM

# Para la prediccion solo tuvimos en cuenta el BMIZ y el HAZ,

In [ ]:
# También se puede tener en cuenta información de la provincia del registro 
g = sns.FacetGrid(data2, col="nombre_provincia",  hue="decae", col_wrap=3)
g = (g.map(plt.scatter, "BMIZ","HAZ",alpha=0.7).add_legend())


In [ ]:
# También se puede tener en cuenta información de si la zona es rural
g = sns.FacetGrid(data2, col="zona_rural",  hue="decae", col_wrap=3)
g = (g.map(plt.scatter, "BMIZ","HAZ",alpha=0.7).add_legend())

# Hacer esto con todas las variables es demasiado tedioso e impreciso
# El Machine Learning viene a hacer este trabajo

<img src="other/images.png">